In [45]:
library(tidyverse)
library(purrr)
library(stringr)
library(magrittr)
library(rworldmap)
library(collections)
library(tuple)
library(lubridate)

### Setting Directories

In [46]:
data_dir <- "C:/Users/Mactuary/OneDrive - UTS/36103/AT2/Data/Timetable"
data_dir

[1] "C:/Users/Mactuary/OneDrive - UTS/36103/AT2/Data/Timetable"

Only those files that end in `.txt` are of interest.

#### Filenames

In [74]:
filepaths <- grep("*.txt", list.files(data_dir), value = TRUE)
filepaths

[1] "agency.txt"         "calendar.txt"       "calendar_dates.txt"
[4] "routes.txt"         "shapes.txt"         "stop_times.txt"    
[7] "stops.txt"          "trips.txt"

In [75]:
data_names <- str_sub(filepaths, 1, -5)
data_names

[1] "agency"         "calendar"       "calendar_dates" "routes"        
[5] "shapes"         "stop_times"     "stops"          "trips"

In [76]:
filepaths <- map(filepaths, function(x) paste(data_dir, x, sep = "/"))
names(filepaths) <- data_names
filepaths

$agency
[1] "C:/Users/Mactuary/OneDrive - UTS/36103/AT2/Data/Timetable/agency.txt"

$calendar
[1] "C:/Users/Mactuary/OneDrive - UTS/36103/AT2/Data/Timetable/calendar.txt"

$calendar_dates
[1] "C:/Users/Mactuary/OneDrive - UTS/36103/AT2/Data/Timetable/calendar_dates.txt"

$routes
[1] "C:/Users/Mactuary/OneDrive - UTS/36103/AT2/Data/Timetable/routes.txt"

$shapes
[1] "C:/Users/Mactuary/OneDrive - UTS/36103/AT2/Data/Timetable/shapes.txt"

$stop_times
[1] "C:/Users/Mactuary/OneDrive - UTS/36103/AT2/Data/Timetable/stop_times.txt"

$stops
[1] "C:/Users/Mactuary/OneDrive - UTS/36103/AT2/Data/Timetable/stops.txt"

$trips
[1] "C:/Users/Mactuary/OneDrive - UTS/36103/AT2/Data/Timetable/trips.txt"

## Defining the Column Types

In [85]:
cols_list <- list(agency = cols(agency_id = col_character(),
                                     agency_name = col_character(),
                                     agency_url = col_character(),
                                     agency_timezone = col_character(),
                                     agency_lang = col_character(),
                                     agency_phone = col_double()),
                  calendar = cols(service_id = col_character(),
                                               monday = col_factor(),
                                               tuesday = col_factor(),
                                               wednesday = col_factor(),
                                               thursday = col_factor(),
                                               friday = col_factor(),
                                               saturday = col_factor(),
                                               sunday = col_factor(),
                                               start_date = col_date(format = "%Y%m%d"),
                                               end_date = col_date(format = "%Y%m%d")),
                  calendar_dates = cols(service_id = col_character(),
                                        date = col_date(format = "%Y%m%d"),
                                        exception_type = col_factor()),
                  routes = cols(route_id = col_character(),
                                agency_id = col_character(),
                                route_short_name = col_character(),
                                route_long_name = col_character(),
                                route_desc = col_factor(),
                                route_type = col_factor(),
                                route_color = col_factor(),
                                route_text_color = col_factor()),
                  shapes = cols(shape_id = col_character(),
                                shape_pt_lat = col_double(),
                                shape_pt_lon = col_double(),
                                shape_pt_sequence = col_integer(),
                                shape_dist_traveled = col_double()),
                  stop_times = cols(trip_id = col_character(),
                                    arrival_time = col_character(),
                                    departure_time = col_character(),
                                    stop_id = col_character(),
                                    stop_sequence = col_integer(),
                                    stop_headsign = col_factor(),
                                    pickup_type = col_factor(),
                                    drop_off_type = col_factor(),
                                    shape_dist_traveled = col_double()),
                  stops = cols(stop_id = col_character(),
                               stop_code = col_double(),
                               stop_name = col_character(),
                               stop_lat = col_double(),
                               stop_lon = col_double(),
                               location_type = col_factor(),
                               parent_station = col_character(),
                               wheelchair_boarding = col_factor(),
                               platform_code = col_factor()),
                  trips = cols(route_id = col_character(),
                               service_id = col_character(),
                               trip_id = col_character(),
                               shape_id = col_character(),  
                               trip_headsign = col_character(),
                               direction_id = col_factor(),
                               block_id = col_character(),
                               wheelchair_accessible = col_factor())
                  )

Create a `tibble` to store the `data`, `filepath` and `column_def`. Note that a `data.frame` won't allow the `col_spec` data type.

In [87]:
files_df <- tibble(data = data_names,
                   filepath = filepaths,
                   column_def = cols_list)
glimpse(files_df)

Observations: 8
Variables: 3
$ data       <chr> "agency", "calendar", "calendar_dates", "routes", "shape...
$ filepath   <list> ["C:/Users/Mactuary/OneDrive - UTS/36103/AT2/Data/Timet...
$ column_def <list> [cols(
  agency_id = col_character(),
  agency_name = c...


In [88]:
data <- map2(files_df$filepath, files_df$column_def, ~readr::read_csv(.x, col_types = .y))

Observations: 127
Variables: 6
$ agency_id       <chr> "701", "700", "x0001", "X0000", "2433", "2434", "24...
$ agency_name     <chr> "train replacement bus operators", "train replaceme...
$ agency_url      <chr> "http://transportnsw.info", "http://transportnsw.in...
$ agency_timezone <chr> "Australia/Sydney", "Australia/Sydney", "Australia/...
$ agency_lang     <chr> "EN", "EN", "EN", "EN", "EN", "EN", "EN", "EN", "EN...
$ agency_phone    <dbl> 131500, 131500, 131500, 131500, 131500, 131500, 131...
Observations: 1,523
Variables: 10
$ service_id <chr> "AA56+1", "AB55+1", "AA76+1", "AA64+1", "AA64+R63+1", "A...
$ monday     <fct> 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0,...
$ tuesday    <fct> 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,...
$ wednesday  <fct> 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,...
$ thursday   <fct> 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,...
$ friday     <fct> 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,

agency_id,agency_name,agency_url,agency_timezone,agency_lang,agency_phone
701,train replacement bus operators,http://transportnsw.info,Australia/Sydney,EN,131500
700,train replacement bus operators,http://transportnsw.info,Australia/Sydney,EN,131500
x0001,Sydney Trains,http://transportnsw.info,Australia/Sydney,EN,131500
X0000,NSW Trains,http://transportnsw.info,Australia/Sydney,EN,131500
2433,Busways Western Sydney,http://transportnsw.info,Australia/Sydney,EN,131500
2434,Interline Bus Services,http://transportnsw.info,Australia/Sydney,EN,131500
2435,Transit Systems,http://transportnsw.info,Australia/Sydney,EN,131500
2436,Hillsbus,http://transportnsw.info,Australia/Sydney,EN,131500
2437,Punchbowl Bus Company,http://transportnsw.info,Australia/Sydney,EN,131500
S0318,Ulladulla Buslines,http://transportnsw.info,Australia/Sydney,EN,131500


In [89]:
names(data) <- data_names
map(data, glimpse)

Observations: 127
Variables: 6
$ agency_id       <chr> "701", "700", "x0001", "X0000", "2433", "2434", "24...
$ agency_name     <chr> "train replacement bus operators", "train replaceme...
$ agency_url      <chr> "http://transportnsw.info", "http://transportnsw.in...
$ agency_timezone <chr> "Australia/Sydney", "Australia/Sydney", "Australia/...
$ agency_lang     <chr> "EN", "EN", "EN", "EN", "EN", "EN", "EN", "EN", "EN...
$ agency_phone    <dbl> 131500, 131500, 131500, 131500, 131500, 131500, 131...
Observations: 1,523
Variables: 10
$ service_id <chr> "AA56+1", "AB55+1", "AA76+1", "AA64+1", "AA64+R63+1", "A...
$ monday     <fct> 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0,...
$ tuesday    <fct> 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,...
$ wednesday  <fct> 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,...
$ thursday   <fct> 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,...
$ friday     <fct> 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,

agency_id,agency_name,agency_url,agency_timezone,agency_lang,agency_phone
701,train replacement bus operators,http://transportnsw.info,Australia/Sydney,EN,131500
700,train replacement bus operators,http://transportnsw.info,Australia/Sydney,EN,131500
x0001,Sydney Trains,http://transportnsw.info,Australia/Sydney,EN,131500
X0000,NSW Trains,http://transportnsw.info,Australia/Sydney,EN,131500
2433,Busways Western Sydney,http://transportnsw.info,Australia/Sydney,EN,131500
2434,Interline Bus Services,http://transportnsw.info,Australia/Sydney,EN,131500
2435,Transit Systems,http://transportnsw.info,Australia/Sydney,EN,131500
2436,Hillsbus,http://transportnsw.info,Australia/Sydney,EN,131500
2437,Punchbowl Bus Company,http://transportnsw.info,Australia/Sydney,EN,131500
S0318,Ulladulla Buslines,http://transportnsw.info,Australia/Sydney,EN,131500


# Handling `arrival_time` and `departure_time` in the `stop_times` table

 When reading the `stop_times` file with `arrival_time` and `departure_time` as `col_time(format = "")`, there are warning messages, e.g. `47 arrival_time   valid date 24:04:00`.
 
 This is explained in the [Buses GTFS Technical Documentation](https://opendata.transport.nsw.gov.au/sites/default/files/TfNSW_Realtime_Bus_Technical_Doc_v2.2.pdf).
>The arrival time at a specific stop for a specific trip on a route. Times for trips starting before 04:00 am will be expressed in ‘36 hour format’. For example: “25:07” (01:07 am)

The data is parsed as `NA` because the built-in function does not parse 36 hour time format.

In [94]:
dummy_stop_times <- readr::read_csv(files_df$filepath[[6]])

Parsed with column specification:
cols(
  trip_id = col_character(),
  arrival_time = col_time(format = ""),
  departure_time = col_time(format = ""),
  stop_id = col_double(),
  stop_sequence = col_double(),
  stop_headsign = col_logical(),
  pickup_type = col_double(),
  drop_off_type = col_double(),
  shape_dist_traveled = col_double()
)
Warning message:
"198579 parsing failures.
row            col   expected   actual                                                                       file
 47 arrival_time   valid date 24:04:00 'C:/Users/Mactuary/OneDrive - UTS/36103/AT2/Data/Timetable/stop_times.txt'
 47 departure_time valid date 24:04:00 'C:/Users/Mactuary/OneDrive - UTS/36103/AT2/Data/Timetable/stop_times.txt'
 48 arrival_time   valid date 24:08:00 'C:/Users/Mactuary/OneDrive - UTS/36103/AT2/Data/Timetable/stop_times.txt'
 48 departure_time valid date 24:08:00 'C:/Users/Mactuary/OneDrive - UTS/36103/AT2/Data/Timetable/stop_times.txt'
 49 arrival_time   valid date 24:14:00 'C:/U

Looking at the string-parsed data (sorted in descending order), there is even an entry with 41 hours, which is beyond the defined 36 hour format. In any case, the strategy is to convert the `arrival_time` and `departure_time` columns to a type that can handle hours greater than 24. The `lubridate::hms` function can handle this.

In [95]:
data$stop_times %>% 
    arrange(desc(arrival_time)) %>%
    head

trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
30.AA89.36-9N-sj2-1.1.R,41:36:00,41:36:00,2500122,3,NA,0,0,37691.03
30.AA89.36-9N-sj2-1.1.R,37:30:00,37:30:00,250019,2,NA,0,0,20497.06
10.AA51.8-790-sj2-2.4.R,34:55:00,34:55:00,254913,9,NA,0,0,244352.31
4.AA88.36-9N-sj2-1.1.R,32:04:00,32:04:00,2500122,3,NA,0,0,37691.03
4.TA.76-621-sj2-1.1.H,31:40:00,31:40:00,22180P1,18,NA,0,0,9515910.84
6.TA.76-621-sj2-1.1.H,31:40:00,31:40:00,22180P1,18,NA,0,0,9515910.84


In [113]:
data$stop_times %<>%
    mutate(arrival_time = hms(arrival_time),
           departure_time = hms(departure_time))

In [116]:
glimpse(data$stop_times)

Observations: 4,722,786
Variables: 9
$ trip_id             <chr> "7.AA56.1-33T-4-sj2-3.1.R", "7.AA56.1-33T-4-sj2...
$ arrival_time        <S4: Period> 22H 16M 0S, 22H 20M 0S, 22H 24M 0S, 22H ...
$ departure_time      <S4: Period> 22H 16M 0S, 22H 20M 0S, 22H 24M 0S, 22H ...
$ stop_id             <chr> "223022", "2230192", "2229196", "2228128", "222...
$ stop_sequence       <int> 1, 2, 3, 4, 5, 6, 7, 1, 2, 3, 4, 5, 6, 7, 1, 2,...
$ stop_headsign       <fct> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA,...
$ pickup_type         <fct> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
$ drop_off_type       <fct> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
$ shape_dist_traveled <dbl> 0.00, 14807.92, 37776.75, 61585.08, 85340.56, 1...


There are only two `time` columns, they are `arrival_time` and `departure_time`. The `readr::read_csv` function default parameters can be modified such that any `time` columns will be parsed as `string`.

Note that another column type `stop_headsign` is not correct, `readr::read_csv` has parsed it as `lgl`.

## Joins

At a glance, there are fields such as `route_id` and `service_id` that occur in multiple tables. These will be used in joining the tables.

In [11]:
comb2 <- combn(names(data), 2)
comb2

agency,agency,agency,agency,agency,agency,agency,calendar,calendar,calendar,...,routes,routes,routes,routes,shapes,shapes,shapes,stop_times,stop_times,stops
calendar,calendar_dates,routes,shapes,stop_times,stops,trips,calendar_dates,routes,shapes,...,shapes,stop_times,stops,trips,stop_times,stops,trips,stops,trips,trips


In [101]:
num_comb = dim(comb2)[2]
# comb_list <- vector(mode = "list", length = num_comb)
# value_list <- vector(mode = "list", length = num_comb)
comb_list <- list()
value_list <- list()
for(column in 1:num_comb){
    comb_list[[column]] <- list(comb2[1, column], comb2[2, column])
    common_fields <- intersect(colnames(data[comb2[1, column]][[1]]), colnames(data[comb2[2, column]][[1]]))
    value_list[[column]] <- ifelse(length(common_fields) == 0, NA, common_fields)
}

In [102]:
key_df <- tibble(combination = comb_list, join = value_list)
key_df

combination,join
"agency , calendar",NA
"agency , calendar_dates",NA
"agency, routes",agency_id
"agency, shapes",NA
"agency , stop_times",NA
"agency, stops",NA
"agency, trips",NA
"calendar , calendar_dates",service_id
"calendar, routes",NA
"calendar, shapes",NA


Not all data combinations have a join.

In [105]:
key_df %<>% filter(!is.na(join))
key_df

combination,join
"agency, routes",agency_id
"calendar , calendar_dates",service_id
"calendar, trips",service_id
"calendar_dates, trips",service_id
"routes, trips",route_id
"shapes , stop_times",shape_dist_traveled
"shapes, trips",shape_id
"stop_times, stops",stop_id
"stop_times, trips",trip_id


The following table shows the fields that are used to join the tables:

|       #       |agency|calendar|calendar_dates|routes|shapes|stop_times|stops|trips|
|---------------|------|--------|--------------|------|------|----------|-----|-----|
|agency         |  -   | #      |  #           |agency_id     |#     |#         |#    |#    |
|calendar       |  #   | -      |service_id    |#     |#     |#         |#    |service_id|
|calendar_dates |  #   | #      |  -           |#     |#     |#         |#    |service_id|
|routes         |  #   | #      |  #           |-     |#     |#         |#    |route_id|
|shapes         |  #   | #      |  #           |#     |-     |shape_dist_traveled|#    |shape_id|
|stop_times     |  #   | #      |  #           |#     |#     |-         |stop_id|trip_id|
|stops          |  #   | #      |  #           |#     |#     |#         |-    |#    |
|trips          |  #   | #      |  #           |#     |#     |#         |#    |-    |

# Calculating the Level of Public Transport In A Given Area

1. The first step is to calculate the number of available services for each `stop_id`.
2. The second step is to classify each `stop_id` into an LGA.
3. The third step is to create some assumptions regarding the "level" of public transport for each mode of transport, e.g. one train vs one bus.
4. The fourth step is to aggregate the scores at an LGA level.

## 1. Counts per stop ID

`stop_id` is a unique identifier for a particular stopping point of a vehicle, with latitude (`stop_lat`) and longitude (`stop_lon`). These are the necessary pieces of information, for each stop:
1. the number of services per day to that stop (count the unique `stop_id` in the `stop_times` table)
2. the type of service, e.g. bus, train, ferry (get the `route_type` for each `stop_id`)
3. for trains, the number of cars for that service
3. the LGA in which the stop exists

In [129]:
data$stops %>%
    left_join(data$stop_times, by = 'stop_id') %>%
    left_join(data$trips, by = 'trip_id') %>%
    left_join(data$routes, by = 'route_id') %>%
    head

stop_id,stop_code,stop_name,stop_lat,stop_lon,location_type,parent_station,wheelchair_boarding,platform_code,trip_id,...,direction_id,block_id,wheelchair_accessible,agency_id,route_short_name,route_long_name,route_desc,route_type,route_color,route_text_color
228055,228055,Pacific Hwy After Murray St,-33.00361,151.6856,NA,NA,0,NA,1.TA.61-266-3-sj2-1.1.H,...,0,NA,2,2452,2663,Hunter Sports HS to North Entrance,School buses,712,00B5EF,FFFFFF
228055,228055,Pacific Hwy After Murray St,-33.00361,151.6856,NA,NA,0,NA,6.TA.61-99-sj2-1.6.H,...,0,NA,1,2452,99,"Charlestown to Lake Haven via Swansea, Gwandalan and Blue Haven",Central Coast Buses Network,700,00B5EF,FFFFFF
228055,228055,Pacific Hwy After Murray St,-33.00361,151.6856,NA,NA,0,NA,12.TA.61-99-sj2-1.11.H,...,0,NA,1,2452,99,"Charlestown to Lake Haven via Swansea, Gwandalan and Blue Haven",Central Coast Buses Network,700,00B5EF,FFFFFF
228055,228055,Pacific Hwy After Murray St,-33.00361,151.6856,NA,NA,0,NA,28.TA.61-99-sj2-1.23.H,...,0,NA,1,2452,99,"Charlestown to Lake Haven via Swansea, Gwandalan and Blue Haven",Central Coast Buses Network,700,00B5EF,FFFFFF
228055,228055,Pacific Hwy After Murray St,-33.00361,151.6856,NA,NA,0,NA,1.TA.73-507-sj2-1.1.H,...,0,NA,1,3000,507,"St. Francis Xavier, Belmont to South St and Pacific Hwy",School buses,712,00B5EF,FFFFFF
228055,228055,Pacific Hwy After Murray St,-33.00361,151.6856,NA,NA,0,NA,1.TA.73-829-sj2-1.1.R,...,1,NA,2,3000,829,Belmont Depot Yard to Whitebridge High School,School buses,712,00B5EF,FFFFFF


In [118]:
df1 <- tibble(x = c(1, 2, 3, 4))
df1

x
1
2
3
4


In [122]:
df2 <- tibble(x = c(1, 2, 2, 3, 3, 3), y = c(1, 2, 3, 4, 5, 6), z = c(10, 20, 30, 40, 50, 60))
df2

x,y,z
1,1,10
2,2,20
2,3,30
3,4,40
3,5,50
3,6,60


In [124]:
df1 %>%
    left_join(df2 %>% select(-z), by = 'x')

x,y
1,1
2,2
2,3
3,4
3,5
3,6
4,NA
